In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#!pip install -U fvcore

In [4]:
#Importing the necessary libraries
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.gta5 import GTA5Custom
from datasets.cityscapes import CityscapesCustom
from models.bisenet.build_bisenet import BiSeNet
from train import train_model
from utils import test_latency_FPS, test_FLOPs_params, plot_miou_over_epochs

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
gta5_height, gta5_width = (720, 1280)
gta5_batch_size = 4

cityscapes_height, cityscapes_width = (512, 1024)
cityscapes_batch_size = 4

n_epochs = 50

In [ ]:
#Create Dataloaders for Cityscapes and GTA5
gta5_dir = os.path.dirname(os.getcwd()) + '/GTA5/GTA5/'
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

gta5_train_dataset = GTA5Custom(gta5_dir, gta5_height, gta5_width, augment=None)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', cityscapes_height, cityscapes_width)

gta5_train_dataloader = DataLoader(gta5_train_dataset, gta5_batch_size, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, cityscapes_batch_size, shuffle=False)

print(f'GTA5 (Train): {len(gta5_train_dataset)} images, divided into {len(gta5_train_dataloader)} batches of size {gta5_train_dataloader.batch_size}')
print(f'Cityscapes (Test): {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

In [ ]:
#Testing Domain Shift problem in semantic segmentation

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Get the class names
class_names = cityscapes_test_dataset.get_class_names()

#Train the model
all_train_miou, all_test_miou, best_epoch = train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            gta5_train_dataloader, cityscapes_test_dataloader, class_names, device, n_epochs, model_name='BiSeNet_shift')

plot_miou_over_epochs(all_train_miou, all_test_miou, best_epoch, model_name='BiSeNet_shift')

In [ ]:
#Create Dataloader for GTA5 with Data Augmentation
augment = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.GaussianBlur(kernel_size=3)
])

gta5_train_dataset_aug = GTA5Custom(gta5_dir, gta5_height, gta5_width, augment)

gta5_train_dataloader_aug = DataLoader(gta5_train_dataset_aug, gta5_batch_size, shuffle=True)

print(f'GTA5 augmented (Train): {len(gta5_train_dataset_aug)} images, divided into {len(gta5_train_dataloader_aug)} batches of size {gta5_train_dataloader_aug.batch_size}')

In [ ]:
#Testing Data Augmentation to mitigate Domain Shift problem

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Train the model
all_train_miou, all_test_miou, best_epoch = train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            gta5_train_dataloader_aug, cityscapes_test_dataloader, class_names, device, n_epochs, model_name='BiSeNet_augment')

plot_miou_over_epochs(all_train_miou, all_test_miou, best_epoch, model_name='BiSeNet_augment')